In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df_character = pd.read_csv(os.getenv('SIMPSONS_CHARACTERS'))
df_lines = pd.read_csv(os.getenv('SIMPSONS_SCRIPT_LINES'), low_memory=False)
df_episode = pd.read_csv(os.getenv('SIMPSONS_EPISODES'))
df_location = pd.read_csv(os.getenv('SIMPSONS_LOCATIONS'))